source: https://github.com/NeuralNine/neuralintents/tree/master

In [4]:
import sys
import threading
import tkinter as tk
from neuralintents import BasicAssistant

#STT
import torch
import sounddevice as sd
import IPython.display as ipd
stt_model_tiny = torch.load('D:/This_season/Tusul/editing_projects/whisper-mongolia/model/whisper-tiny-mn-9.pt')
stt_model_small = torch.load('D:/This_season/Tusul/editing_projects/whisper-mongolia/model/whisper-small-v12-mn.pt')
#stt_model_large = torch.load('D:/This_season/Tusul/editing_projects/whisper-mongolia/model/whisper-large-v2-mn-13.pt')

# TTS
from transformers import VitsModel, AutoTokenizer
from IPython.display import Audio
tts_model = VitsModel.from_pretrained("facebook/mms-tts-mon")
tokenizer = AutoTokenizer.from_pretrained("facebook/mms-tts-mon")

from io import BytesIO
import wave
import simpleaudio as sa
import time
def mms_tts(text):
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        output = tts_model(**inputs).waveform

    scaled, nchan = Audio._validate_and_normalize_with_numpy(output, normalize=True)

    fp = BytesIO()
    waveobj = wave.open(fp,mode='wb')
    waveobj.setnchannels(nchan)
    waveobj.setframerate(16000)
    waveobj.setsampwidth(2)
    waveobj.setcomptype('NONE','NONE')
    waveobj.writeframes(scaled)
    val = fp.getvalue()
    waveobj.close()
    return val , output.shape[1]

def stt(duration=3):
        sample_rate = 16000
        recorded_audio = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1, blocking=True)
        print('Асуултыг боловсруулаж байна.')
        text = stt_model_small(recorded_audio[:,0])["text"]
        return text

Some weights of the model checkpoint at facebook/mms-tts-mon were not used when initializing VitsModel: ['flow.flows.3.wavenet.in_layers.3.weight_g', 'flow.flows.0.wavenet.in_layers.2.weight_v', 'flow.flows.1.wavenet.in_layers.1.weight_g', 'posterior_encoder.wavenet.in_layers.8.weight_v', 'posterior_encoder.wavenet.res_skip_layers.3.weight_g', 'posterior_encoder.wavenet.res_skip_layers.13.weight_g', 'posterior_encoder.wavenet.res_skip_layers.5.weight_v', 'posterior_encoder.wavenet.res_skip_layers.9.weight_g', 'posterior_encoder.wavenet.res_skip_layers.12.weight_g', 'flow.flows.0.wavenet.res_skip_layers.3.weight_g', 'flow.flows.0.wavenet.res_skip_layers.1.weight_g', 'flow.flows.3.wavenet.in_layers.2.weight_v', 'flow.flows.1.wavenet.res_skip_layers.1.weight_g', 'flow.flows.0.wavenet.in_layers.3.weight_v', 'flow.flows.1.wavenet.in_layers.3.weight_v', 'flow.flows.0.wavenet.in_layers.0.weight_g', 'flow.flows.1.wavenet.res_skip_layers.3.weight_v', 'posterior_encoder.wavenet.res_skip_layers.0

In [5]:
class Assistant:

    def __init__(self):

        self.assistant = BasicAssistant("mn_intents.json")
        self.data_changed = False
        if self.data_changed == True:
            self.assistant.fit_model(epochs=50)
            self.assistant.save_model()
        else:
             self.assistant.load_model()

        self.root = tk.Tk()
        self.label = tk.Label(text="😊", font=("Arial", 120, "bold"))
        self.label.pack()

        threading.Thread(target=self.run_assistant).start()

        self.root.mainloop()
    
    def create_file(self):
        with open("New_file.txt", "w") as f:
            f.write("шинэ файлыг амжилттай үүсэглээ.")

    def run_assistant(self):
        while True:
            try:
                #text = input("Enter a message: ")
                print('Асуултаа хэлнэ үү?')
                text = stt(duration=3)
                text = text.lower()
                print(text)

                if "байна уу" in text:
                    self.label.config(fg="red")

                elif text == "зогс":
                            self.root.destroy()
                            sys.exit(0)
                elif text is not None and len(text) > 3:
                            response = self.assistant.process_input(text)
                            print("Хариулт " + f'{response}')
                            if response == "new_text_file":
                                self.create_file()
                                audio, siz = mms_tts(text = 'файлыг амжилттай үүсэглээ')
                                play_obj = sa.play_buffer(audio, 1, 2, 16000)
                                time.sleep(siz/16000)
                            if response is not None and len(response) > 3:
                                audio, siz = mms_tts(text = response)
                                play_obj = sa.play_buffer(audio, 1, 2, 16000)
                                time.sleep(siz/16000)
                self.label.config(fg="black")
            except:
                self.label.config(fg="black")
                continue

Assistant()

Асуултаа хэлнэ үү?
Асуултыг боловсруулаж байна.
сайн байна уу
Асуултаа хэлнэ үү?
Асуултыг боловсруулаж байна.
боловсруулан тэгэхээр виспер модуль маань а энд би сайн байна уу
Асуултаа хэлнэ үү?
Асуултыг боловсруулаж байна.
таван асуулт байна а зүгээр суутган
Хариулт сайн байна уу эрхэмээ, танийг сонсож байна. таньд юугаар туслах вэ
Асуултаа хэлнэ үү?
Асуултыг боловсруулаж байна.
отгон баатар гэж хэн бэ
Хариулт номин-эрдэнэ нь монгол улсын их сургуульд сүлжээний технологи мэргэжилээр сурдаг. тэр отгонбаатартай хамтран төслийн ажил хичээлийн хүрээнд намайг хөгжүүлэж байгаа.
Асуултаа хэлнэ үү?
Асуултыг боловсруулаж байна.
шинэ файл үүсгээрэй
Хариулт new_text_file
Асуултаа хэлнэ үү?
Асуултыг боловсруулаж байна.
та чинь хамаа өгсөн юм байна шүү дээ
Хариулт сайн байна уу эрхэмээ, танийг сонсож байна. таньд юугаар туслах вэ
Асуултаа хэлнэ үү?


KeyboardInterrupt: 

Асуултыг боловсруулаж байна.
та гэдгээ яаж үүсгэсэн бэ гээд тэгэхээр файл гэсэн төрөл дээр би
Хариулт new_text_file


Exception in thread Thread-6 (run_assistant):
Traceback (most recent call last):
  File "C:\Users\Oggy\AppData\Local\Temp\ipykernel_12044\2803705264.py", line 52, in run_assistant
  File "c:\Users\Oggy\anaconda3\Lib\tkinter\__init__.py", line 1702, in configure
    return self._configure('configure', cnf, kw)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Oggy\anaconda3\Lib\tkinter\__init__.py", line 1692, in _configure
    self.tk.call(_flatten((self._w, cmd)) + self._options(cnf))
RuntimeError: main thread is not in main loop

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\Oggy\anaconda3\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "c:\Users\Oggy\anaconda3\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Oggy\AppData\Local\Temp\ipykernel_12044\2803705264.py", line 54, in run_assistant
  File "c:\Users\Oggy\anaconda3\